In [2]:
using Pkg
Pkg.activate()

  Activating project at `~/.julia/environments/v1.7`


In [25]:
using SparseArrays

In [4]:
using CPDTGW3D8a
using ViewerGL, LinearAlgebra
GL = ViewerGL
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation


#include("")
store = [];
scaling = 1.5;
V,(VV,EV,FV,CV) = Lar.cuboid([0.25,0.25,0.25],true,[-0.25,-0.25,-0.25]);
mybox = (V,CV,FV,EV);
for k=1:5
	size = rand()*scaling
	scale = Lar.s(size,size,size)
	transl = Lar.t(rand(3)...)
	alpha = 2*pi*rand()
	rx = Lar.r(alpha,0,0); ry = Lar.r(0,alpha,0); rz = Lar.r(0,0,alpha)
	rot = rx * ry * rz
	str = Lar.Struct([ transl, scale, rot, mybox ])
	obj = Lar.struct2lar(str)
	vs = obj[1]
	diag = LinearAlgebra.norm(vs[:,8]-vs[:,1])
	if diag > 1/5
		push!(store, obj)
	end
end

str = Lar.Struct(store);
V,CV,FV,EV = Lar.struct2lar(str);
# V = Plasm.normalize3D(V) TODO:  solve MethodError bug

GL.VIEW([ GL.GLPol(V,CV, GL.COLORS[2], 0.1) ]);

function testarrangement(V,CV,FV,EV)
		cop_EV = Lar.coboundary_0(EV::Lar.Cells);
		cop_FE = Lar.coboundary_1(V, FV::Lar.Cells, EV::Lar.Cells);
		W = convert(Lar.Points, V');

		V, copEV, copFE, copCF = CPDTGW3D8a.spatial_arrangement(
				W::Lar.Points, cop_EV::Lar.ChainOp, cop_FE::Lar.ChainOp);

		V = convert(Lar.Points, V');
		V,CVs,FVs,EVs = Lar.pols2tria(V, copEV, copFE, copCF) # whole assembly
		GL.VIEW(GL.GLExplode(V,FVs,1.1,1.1,1.1,99,1));
		GL.VIEW(GL.GLExplode(V,EVs,1.5,1.5,1.5,99,1));
		GL.VIEW(GL.GLExplode(V,CVs,1,1,1,99,0.2));
end

#testarrangement(V,CV,FV,EV);

In [17]:
cop_EV = Lar.coboundary_0(EV::Lar.Cells);
cop_FE = Lar.coboundary_1(V, FV::Lar.Cells, EV::Lar.Cells);
W = convert(Lar.Points, V');

In [6]:
cop_EV

60×40 SparseArrays.SparseMatrixCSC{Int8, Int64} with 120 stored entries:
⠉⠒⠤⣀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠑⠑⢄⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠑⢄⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠉⠒⠤⣀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠑⠑⢄⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠑⢄⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠉⠒⠤⣀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠑⠑⢄⢄⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠉⠒⠤⣀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⠑⢄⢄⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠑⢄⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠉⠒⠤⣀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⠑⢄⢄
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠑⢄

In [7]:
cop_FE

30×60 SparseArrays.SparseMatrixCSC{Int8, Int64} with 120 stored entries:
⢍⢖⠉⠒⠤⣀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠑⠑⠑⠑⠤⣀⠤⣀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠑⠑⢄⢄⢍⢖⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢍⢖⠉⠒⠤⣀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⠑⠑⠑⠤⣀⠤⣀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⠑⢄⢄⢍⢖⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢍⢖⠉⠒⠤⣀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⠑⠑⠑

In [8]:
W

40×3 Matrix{Float64}:
  0.267569    0.791765   0.103698
  0.296773    0.762672   0.435564
  0.238476    1.12341    0.135331
  0.26768     1.09432    0.467198
  0.599435    0.823399   0.0772665
  0.62864     0.794306   0.409133
  0.570342    1.15504    0.1089
  0.599547    1.12595    0.440767
  0.0622929  -0.189704   0.516424
  0.418067   -0.494621   1.02363
 -0.242624    0.223059   0.978444
  0.11315    -0.0818581  1.48565
  0.5695      0.272316   0.438401
  ⋮                      
  0.62257     0.251934   0.972151
  0.536798    0.319011   1.05632
  0.689646    0.369416   0.946879
  0.603874    0.436492   1.03104
  0.3287      0.163196   0.277191
  0.191483    0.24952    0.347054
  0.415023    0.31597    0.257968
  0.277806    0.402293   0.327831
  0.398563    0.143974   0.438158
  0.261346    0.230297   0.508021
  0.484886    0.296748   0.418936
  0.347669    0.383071   0.488799

In [9]:
V, copEV, copFE, copCF = CPDTGW3D8a.spatial_arrangement(W::Lar.Points, cop_EV::Lar.ChainOp, cop_FE::Lar.ChainOp);

LoadError: BoundsError: attempt to access 0×0 SparseArrays.SparseMatrixCSC{Int8, Int64} with 0 stored entries at index [:, 1]

In [10]:
V, copEV, copFE, copCF = Lar.Arrangement.spatial_arrangement(W::Lar.Points, cop_EV::Lar.ChainOp, cop_FE::Lar.ChainOp);

In [13]:
rV, rcopEV, rcopFE = CPDTGW3D8a.spatial_arrangement_1(V,copEV,copFE,false)

(Matrix{Float64}(undef, 0, 3), sparse(Int64[], Int64[], Int8[], 0, 0), sparse(Int64[], Int64[], Int8[], 0, 0))

In [14]:
rcopEV

0×0 SparseArrays.SparseMatrixCSC{Int8, Int64} with 0 stored entries

In [16]:
rV, rcopEV, rcopFE = Lar.Arrangement.spatial_arrangement_1(V,copEV,copFE,false)

([0.26756859999999993 0.7917651000000001 0.10369780000000002; 0.29677289999999995 0.7626724000000003 0.4355643; … ; 0.4848857999999999 0.2967476000000001 0.4189359999999999; 0.34766943608910744 0.38307079007035877 0.48879908315084514], sparse([1, 3, 9, 1, 4, 10, 2, 3, 11, 2  …  65, 61, 64, 66, 62, 63, 67, 62, 64, 68], [1, 1, 1, 2, 2, 2, 3, 3, 3, 4  …  41, 42, 42, 42, 43, 43, 43, 44, 44, 44], Int8[-1, -1, -1, 1, -1, -1, -1, 1, -1, 1  …  1, 1, -1, 1, -1, 1, 1, 1, 1, 1], 68, 44), sparse([1, 3, 1, 4, 1, 5, 1, 6, 2, 3  …  31, 35, 32, 34, 32, 35, 33, 34, 33, 35], [1, 1, 2, 2, 3, 3, 4, 4, 5, 5  …  64, 64, 65, 65, 66, 66, 67, 67, 68, 68], Int8[1, 1, -1, 1, -1, 1, 1, 1, 1, -1  …  1, -1, -1, -1, 1, -1, -1, 1, 1, 1], 35, 68))

In [18]:
FV = Lar.compute_FV( copEV, copFE )

35-element Vector{Vector{Int64}}:
 [1, 2, 3, 4]
 [5, 6, 7, 8]
 [1, 2, 5, 6]
 [3, 4, 7, 8]
 [1, 3, 5, 7]
 [2, 4, 6, 8]
 [9, 10, 11, 12]
 [13, 14, 15, 16, 17, 18, 19, 20]
 [17, 18, 19, 20]
 [9, 10, 13, 14]
 [11, 12, 15, 16]
 [9, 11, 13, 15]
 [10, 12, 14, 16]
 ⋮
 [29, 30, 33, 34]
 [31, 32, 35, 36]
 [18, 19, 29, 33]
 [18, 19, 31, 35]
 [17, 20, 30, 34]
 [17, 20, 32, 36]
 [37, 38, 39, 40]
 [41, 42, 43, 44]
 [37, 38, 41, 42]
 [39, 40, 43, 44]
 [37, 39, 41, 43]
 [38, 40, 42, 44]

In [19]:
model = (convert(Lar.Points,V'), FV)

([0.2675686 0.2967729 … 0.4848857999999999 0.3476694360891075; 0.7917651 0.7626724000000001 … 0.2967476000000001 0.38307079007035877; 0.1036978 0.4355643 … 0.4189359999999999 0.4887990831508452], [[1, 2, 3, 4], [5, 6, 7, 8], [1, 2, 5, 6], [3, 4, 7, 8], [1, 3, 5, 7], [2, 4, 6, 8], [9, 10, 11, 12], [13, 14, 15, 16, 17, 18, 19, 20], [17, 18, 19, 20], [9, 10, 13, 14]  …  [18, 19, 29, 33], [18, 19, 31, 35], [17, 20, 30, 34], [17, 20, 32, 36], [37, 38, 39, 40], [41, 42, 43, 44], [37, 38, 41, 42], [39, 40, 43, 44], [37, 39, 41, 43], [38, 40, 42, 44]])

In [20]:
sp_idx = Lar.spaceindex(model) # OK!!  tested symmetry of computed relation

35-element Vector{Vector{Int64}}:
 [5, 3, 4, 6]
 [5, 3, 4, 6, 14]
 [5, 2, 1, 6, 14]
 [5, 2, 1, 6]
 [3, 2, 1, 4]
 [3, 2, 1, 4, 14, 17]
 [12, 10, 8, 11, 13]
 [33, 32, 35, 31, 14, 16, 12, 10, 15, 7  …  20, 26, 9, 24, 11, 29, 28, 13, 23, 22]
 [12, 8, 21, 27, 25, 20, 26, 24, 11, 29, 28, 13, 23, 22]
 [32, 35, 31, 12, 8, 7, 26, 24, 11, 13, 22]
 [12, 10, 8, 7, 21, 27, 25, 20, 26, 9, 24, 29, 28, 13, 23, 22]
 [33, 32, 35, 31, 10, 8, 7, 21, 25, 20, 26, 9, 24, 11, 29, 28, 13, 23, 22]
 [12, 10, 8, 7, 27, 25, 20, 26, 9, 24, 11, 29, 28, 23, 22]
 ⋮
 [12, 10, 8, 21, 27, 20, 26, 9, 11, 28, 13, 23, 22]
 [12, 8, 21, 27, 20, 9, 11, 29, 28, 13, 23, 22]
 [12, 10, 8, 21, 27, 20, 9, 24, 11, 28, 13, 23, 22]
 [8, 21, 25, 20, 26, 9, 24, 11, 29, 13, 23, 22]
 [12, 8, 21, 25, 20, 26, 9, 24, 11, 29, 13, 23, 22]
 [12, 8, 21, 27, 25, 20, 9, 11, 28, 13, 23, 22]
 [34, 33, 32, 35]
 [34, 33, 32, 35, 12, 10, 8]
 [30, 34, 35, 31, 12, 10, 8]
 [30, 34, 35, 31, 12, 8]
 [30, 33, 32, 35, 31]
 [30, 34, 33, 32, 31, 12, 10, 8]

In [21]:
fs_num = size(copFE, 1)

35

In [22]:
rV = Array{Float64,2}(undef,0,3)

0×3 Matrix{Float64}

In [26]:
rEV = SparseArrays.spzeros(Int8,0,0)

0×0 SparseMatrixCSC{Int8, Int64} with 0 stored entries

In [27]:
rFE = SparseArrays.spzeros(Int8,0,0)

0×0 SparseMatrixCSC{Int8, Int64} with 0 stored entries

In [29]:
function frag_face(V::Matrix{Float64}, EV::SparseMatrixCSC{Int8, Int64}, FE::SparseMatrixCSC{Int8, Int64},
    sp_idx::Vector{Int64}, sigma::Int64)


    vs_num::Int64 = size(V, 1)

	# 2D transformation of sigma face
    sigmavs::Vector{Int64} = (abs.(FE[sigma:sigma,:])*abs.(EV))[1,:].nzind

    sV::Matrix{Float64} = V[sigmavs, :]
    sEV::SparseMatrixCSC{Int8, Int64} = EV[FE[sigma, :].nzind, sigmavs]
    M::Matrix{Float64} = Lar.Arrangement.submanifold_mapping(sV)
   @views tV::Matrix{Float64} = ([V ones(vs_num)]*M)[:, 1:3]
    sV = tV[sigmavs, :]

    # sigma face intersection with faces in sp_idx[sigma]
    @async for i in sp_idx[sigma]
        tmpV::Matrix{Any}, tmpEV::SparseMatrixCSC{Int8, Int64} = Lar.Arrangement.face_int(tV, EV, FE[i, :])
        sV, sEV = Lar.skel_merge(sV, sEV, tmpV, tmpEV)
    end

    # computation of 2D arrangement of sigma face
    sV = sV[:, 1:2]
    nV::Matrix{Float64}, nEV::SparseMatrixCSC{Int8, Int64}, nFE::SparseMatrixCSC{Int8, Int64} = planar_arrangement(sV, sEV, sparsevec(ones(Int8, length(sigmavs))))

    nvsize::Int64 = size(nV, 1)
    nV = [nV zeros(nvsize) ones(nvsize)]*inv(M)[:, 1:3]

    return nV, nEV, nFE
end

frag_face (generic function with 1 method)

In [30]:
@async for sigma in 1:fs_num
            #println("\n",sigma, "/", fs_num)
            nV, nEV, nFE = frag_face(V, copEV, copFE, sp_idx, sigma)
		      	nV = convert(Lar.Points, nV)
            rV,rEV, rFE= Lar.skel_merge( rV,rEV,rFE,  nV,nEV,nFE )
            #rV=a;  #rEV=b;  #rFE=c
        end

Task (failed) @0x000000010f100180
MethodError: no method matching frag_face(::Matrix{Float64}, ::SparseMatrixCSC{Int8, Int64}, ::SparseMatrixCSC{Int8, Int64}, ::Vector{Vector{Int64}}, ::Int64)
Closest candidates are:
  frag_face(::Matrix{Float64}, ::SparseMatrixCSC{Int8, Int64}, ::SparseMatrixCSC{Int8, Int64}, ::Vector{Int64}, ::Int64) at In[29]:1
Stacktrace:
 [1] macro expansion
   @ ./In[30]:3 [inlined]
 [2] (::var"#5#6")()
   @ Main ./task.jl:423

In [ ]:
# spaceindex computation
	FV = Lar.compute_FV( copEV, copFE )
	model = (convert(Lar.Points,V'), FV)
	sp_idx = Lar.spaceindex(model) # OK!!  tested symmetry of computed relation

	# initializations
	fs_num = size(copFE, 1)
  rV = Array{Float64,2}(undef,0,3)
  rEV = SparseArrays.spzeros(Int8,0,0)
  rFE = SparseArrays.spzeros(Int8,0,0)

	# multiprocessing of face fragmentation
    if (multiproc == true)
        in_chan = Distributed.RemoteChannel(()->Channel{Int64}(0))
        out_chan = Distributed.RemoteChannel(()->Channel{Tuple}(0))
        @async begin
            for sigma in 1:fs_num
                put!(in_chan, sigma)
            end
            for p in Distributed.workers()
                put!(in_chan, -1)
            end
        end
        for p in Distributed.workers()
            @async Base.remote_do(
                frag_face_channel, p, in_chan, out_chan, V, EV, FE, sp_idx)
        end
        for sigma in 1:fs_num
            rV, rEV, rFE = Lar.skel_merge(rV, rEV, rFE, take!(out_chan)...)
        end
    else
	# sequential (iterative) processing of face fragmentation
        @async for sigma in 1:fs_num
            #println("\n",sigma, "/", fs_num)
            nV, nEV, nFE = frag_face(V, copEV, copFE, sp_idx, sigma)
		      	nV = convert(Lar.Points, nV)
            rV,rEV, rFE= Lar.skel_merge( rV,rEV,rFE,  nV,nEV,nFE )
            #rV=a;  #rEV=b;  #rFE=c
        end
    end
	# merging of close vertices, edges and faces (3D congruence)
	rV, rEV, rFE = merge_vertices(rV, rEV, rFE)
  return rV, rEV, rFE
end